Import library

In [0]:
from math import exp
from random import seed
from random import random

In [0]:
import pandas as pd
import numpy as np

import file

In [0]:
path = '/content/ypred (1).csv'
df = pd.read_csv(path)

Train test split

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

In [8]:
data = np.array(df)
data = normalize(data)
X = data[:, 0:5]
y = data[:, 5]
y = list(y)
y = [[i] for i in y]
print((y))
X = list(X)
X = [list(i) for i in X]

[[0.9486832514125672], [0.9863938714085463], [0.9938836810467544], [0.9965457041859566], [0.9958931302203653], [0.9701422055716726], [0.9899493330613401], [0.99503707155059], [0.9938835648552448], [0.996545628165611], [0.9805800638454979], [0.992277561512923], [0.9899490361952004], [0.9950369093020923], [0.9970542807881283], [0.9863930260791912], [0.9805792944051533], [0.9922772275505571], [0.9958928031596114], [0.9974584098476728], [0.9486781212804194], [0.986392298704509], [0.9938829064374588], [0.9965452311707554], [0.9958925282683068], [0.9701377168633949], [0.9899476784010142], [0.9950361997685501], [0.9938824327349745], [0.9965449524299702], [0.9805806200685618], [0.9922778241801893], [0.9899494255278973], [0.9950371272478536], [0.9970544249525188], [0.9863937469025358], [0.9805804068496088], [0.9922777153608681], [0.9958930840706688], [0.9974585965320186], [0.9486818522773639], [0.9863934127024251], [0.9938834427052399], [0.9965455521452827], [0.9958929295695578], [0.97014066257

network

In [0]:
# Initialize a network
def initialize_network(n_inputs, n_hidden1, n_hidden2, n_outputs):
	network = list()
	hidden_layer1 = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden1)]
	network.append(hidden_layer1)
	print("hidden layer1")
	print(hidden_layer1)
 
	hidden_layer2 = [{'weights':[random() for i in range(n_hidden1 + 1)]} for i in range(n_hidden2)]
	network.append(hidden_layer2)
	print("hidden layer2")
	print(hidden_layer2)
 
	output_layer = [{'weights':[random() for i in range(n_hidden2 + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	print("output layer")
	print(output_layer)
	return network

In [0]:
# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation

In [0]:
# Transfer neuron activation
def transfer(activation, name_func):
	if name_func == 'relu':
		if activation > 0:
			return activation
		else:
			return 0
	elif name_func == 'identity':
		return (activation)
	elif name_func == 'sigmoid':
		return 1.0 / (1.0 + exp(-activation))
	elif name_func == 'tanh':
		return (exp(activation) - exp(-activation)) / (exp(activation) + exp(-activation))

In [0]:
# Forward propagate input to a network output
def forward_propagate(network, row, func_name):
	inputs = row
	for num_layer in range(len(network)):
		new_inputs = []
		for neuron in network[num_layer]:
			activation = activate(neuron['weights'], inputs)
			# print('activation', activation)
			neuron['output'] = transfer(activation, func_name[num_layer])
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs

In [0]:
# Calculate the derivative of an neuron output
def transfer_derivative(output, name_func):
	if name_func == 'relu':
		if output > 0:
			return 1
		else:
			return 0
	elif name_func == 'identity':
		return 1
	elif name_func == 'sigmoid':
		return output * (1.0 - output)
	elif name_func == 'tanh':
		return 1 - output**2

In [0]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected, func_name):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		#if we're in hidden layer
		if i != len(network)-1:
			#print("in hidden ")
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
				
		#else we're in output layer
		else:
			#print("in output")
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		#for both hidden and output
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'], func_name[i])
			#print("delta")
			#print(neuron['delta'])

In [0]:
# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
				#print('>epoch=%.3f, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
			neuron['weights'][-1] += l_rate * neuron['delta']
			#print("neuron weights")
			#print(neuron['weights'])

In [0]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs, output, func_name):
  counter = 1
  for epoch in range(n_epoch) :
    sum_error = 0
    for row in range(len(train)):
      outputs = forward_propagate(network, train[row], func_name)
      expected = output[row]
      #sum_error += sum([(expected[i]-outputs[i])**2  for i in range(len(expected))])
      #print('expected[0] - outputs[0])', expected,  outputs[0])
      sum_error += (expected[0] - outputs[0])**2
      #if counter == 1:
        #print('sum_error', sum_error)
      backward_propagate_error(network, expected, func_name)
      update_weights(network, train[row], l_rate)
    counter = counter + 1
    print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [0]:
# Test training backprop algorithm
n_inputs = len(X[0]) 
n_outputs = 1 
network = initialize_network(n_inputs, 32, 16, n_outputs)
train_network(network, X, 0.5, 20, n_outputs, y, ['relu', 'relu', 'identity'])


hidden layer1
[{'weights': [0.7165650247854929, 0.9039706136283984, 0.8471789028100756, 0.6709097519718685, 0.09821885590606849, 0.46197276315093994]}, {'weights': [0.3430279231818649, 0.24895004799516474, 0.3364316063561622, 0.8652292310685062, 0.4434250298440249, 0.8627232313253677]}, {'weights': [0.30030306406816965, 0.30053886462029455, 0.922434384247626, 0.636704231201048, 0.09467864742741872, 0.9059420666631534]}, {'weights': [0.9252467441581248, 0.5297422255575895, 0.44547594985450933, 0.36904782895398636, 0.40854563521257736, 0.6640227660001774]}, {'weights': [0.19437383924240048, 0.5461590886336577, 0.34811812185105107, 0.20290156489376776, 0.48097310873974086, 0.32836661291397085]}, {'weights': [0.5664656176939462, 0.8523077882169092, 0.5115327257598763, 0.2932303148066072, 0.5292518415181747, 0.11687057578445292]}, {'weights': [0.5186384414789333, 0.17474940990349286, 0.24446637415257577, 0.5749878209591147, 0.07752628981019516, 0.43167453574369974]}, {'weights': [0.37258287

In [0]:
print('Predict:', forward_propagate(network, X[7], ['relu', 'relu', 'identity']))
print('True:', y[7])

Predict: [0.9905789985877751]
True: [0.99503707155059]


In [0]:
network

tests

In [0]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs, output, func_name):
  counter = 1
  for epoch in range(n_epoch) :
    sum_error = 0
    for row in range(len(train)):
      outputs = forward_propagate(network, train[row], func_name)
      expected = output[row]
      #sum_error += sum([(expected[i]-outputs[i])**2  for i in range(len(expected))])
      #print('expected[0] - outputs[0])', expected,  outputs[0])
      sum_error += (expected[0] - outputs[0])**2
      #if counter == 1:
        #print('sum_error', sum_error)
      backward_propagate_error(network, expected, func_name)
      update_weights(network, train[row], l_rate)
    counter = counter + 1
    print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
    print('\n', network[1])

In [0]:
n_inputs = len(X[0]) 
n_outputs = 1 
network = initialize_network(n_inputs, 32, 20, n_outputs)
train_network(network, X, 0.5, 10, n_outputs, y, ['relu', 'relu', 'identity'])


hidden layer1
[{'weights': [0.7684632145434748, 0.7634202343832774, 0.6019962299820789, 0.990313813073937, 0.09538567931584985, 0.3929650938090248]}, {'weights': [0.4954079154181784, 0.5729785839236955, 0.24328777000243984, 0.9921778815084114, 0.19684600122590712, 0.343451454487952]}, {'weights': [0.030005351079272158, 0.3787303256127371, 0.48378182237617884, 0.3755892787309426, 0.9889307276229777, 0.005271881500480968]}, {'weights': [0.9655561294744314, 0.37823797424141536, 0.3576343955321053, 0.8684071707828036, 0.6820041099495893, 0.240028476381567]}, {'weights': [0.06532388950662493, 0.04919258003113802, 0.3387555849840128, 0.7129703474756742, 0.26561499144086564, 0.762346511491316]}, {'weights': [0.6635267199756084, 0.9080869710565328, 0.565375491855689, 0.04398890860259941, 0.8349238025436005, 0.14137791880072348]}, {'weights': [0.8894123146058451, 0.7083588242190944, 0.3133370227347554, 0.5372361181498964, 0.006384532564503775, 0.42526146613003957]}, {'weights': [0.3302144330632

In [0]:
network

Class Custome network

In [0]:
class CustomeNetwork():
	def __init__(self, n_inputs, n_hidden1, n_hidden2, n_outputs, f_activation):
		self.network = list()
		hidden_layer1 = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden1)]
		self.network.append(hidden_layer1)
		print("hidden layer1")
		print(hidden_layer1)
  
		hidden_layer2 = [{'weights':[random() for i in range(n_hidden1 + 1)]} for i in range(n_hidden2)]
		self.network.append(hidden_layer2)
		print("hidden layer2")
		print(hidden_layer2)
 
		output_layer = [{'weights':[random() for i in range(n_hidden2 + 1)]} for i in range(n_outputs)]
		self.network.append(output_layer)
		print("output layer")
		print(output_layer)
		self.f_activation = f_activation

  # Calculate neuron activation for an input
	def activate(self, weights, inputs):
		activation = weights[-1]
		for i in range(len(weights)-1):
			activation += weights[i] * inputs[i]
		return activation

  # Transfer neuron activation
	def transfer(self, activation, name_func):
		if name_func == 'relu':
			if activation > 0:
				return activation
			else:
				return 0
		elif name_func == 'identity':
			return (activation)
		elif name_func == 'sigmoid':
			return 1.0 / (1.0 + exp(-activation))
		elif name_func == 'tanh':
			return (exp(activation) - exp(-activation)) / (exp(activation) + exp(-activation))

  # Forward propagate input to a network output
	def forward_propagate(self, row, func_name):
		inputs = row
		for num_layer in range(len(self.network)):
			new_inputs = []
			for neuron in self.network[num_layer]:
				activation = self.activate(neuron['weights'], inputs)
				# print('activation', activation)
				neuron['output'] = self.transfer(activation, func_name[num_layer])
				new_inputs.append(neuron['output'])
			inputs = new_inputs
		return inputs

  # Calculate the derivative of an neuron output
	def transfer_derivative(self, output, name_func):
		if name_func == 'relu':
			if output > 0:
				return 1
			else:
				return 0
		elif name_func == 'identity':
			return 1
		elif name_func == 'sigmoid':
			return output * (1.0 - output)
		elif name_func == 'tanh':
			return 1 - output**2

  # Backpropagate error and store in neurons
	def backward_propagate_error(self, expected, func_name):
		for i in reversed(range(len(self.network))):
			layer = self.network[i]
			errors = list()
			#if we're in hidden layer
			if i != len(self.network)-1:
				#print("in hidden ")
				for j in range(len(layer)):
					error = 0.0
					for neuron in self.network[i + 1]:
						error += (neuron['weights'][j] * neuron['delta'])
					errors.append(error)				
		  #else we're in output layer
			else:
			  #print("in output")
				for j in range(len(layer)):
					neuron = layer[j]
					errors.append(expected[j] - neuron['output'])
		  #for both hidden and output
			for j in range(len(layer)):
				neuron = layer[j]
				neuron['delta'] = errors[j] * self.transfer_derivative(neuron['output'], func_name[i])
			  #print("delta")
			  #print(neuron['delta'])

  # Update network weights with error
	def update_weights(self, row, l_rate):
		for i in range(len(self.network)):
			inputs = row[:]
			if i != 0:
				inputs = [neuron['output'] for neuron in self.network[i - 1]]
			for neuron in self.network[i]:
				for j in range(len(inputs)):
					neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
				  #print('>epoch=%.3f, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
				neuron['weights'][-1] += l_rate * neuron['delta']
			  #print("neuron weights")
			  #print(neuron['weights'])

  # Train a network for a fixed number of epochs
	def train_network(self, train, l_rate, n_epoch, n_outputs, output):
		network = self.network
		func_name = self.f_activation
		counter = 1
		for epoch in range(n_epoch) :
			sum_error = 0
			for row in range(len(train)):
				outputs = self.forward_propagate(train[row], func_name)
				expected = output[row]
        #sum_error += sum([(expected[i]-outputs[i])**2  for i in range(len(expected))])
        #print('expected[0] - outputs[0])', expected,  outputs[0])
				sum_error += (expected[0] - outputs[0])**2
        #if counter == 1:
          #print('sum_error', sum_error)
				self.backward_propagate_error( expected, func_name)
				self.update_weights( train[row], l_rate)
			counter = counter + 1
			print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
  


In [29]:
# Test training backprop algorithm
n_inputs = len(X[0]) 
n_outputs = 1 
network = CustomeNetwork(n_inputs, 32, 16, n_outputs,  ['relu', 'relu', 'identity'])
network.train_network( X, 0.5, 20, n_outputs, y)

hidden layer1
[{'weights': [0.7292143451986205, 0.5981394078745877, 0.1447037872093998, 0.6199560858042708, 0.17529576467343166, 0.6331707475972281]}, {'weights': [0.30315076659998197, 0.35569229866136365, 0.07243468935488206, 0.23298004751539025, 0.1229065539787797, 0.42004807990088655]}, {'weights': [0.25665583505764666, 0.7092923917676864, 0.2594149007319787, 0.7463021358341975, 0.4539477682871532, 0.7248927811946795]}, {'weights': [0.8439169706967015, 0.37194038132985285, 0.6617604073096129, 0.5192460239923375, 0.5859194067911275, 0.7990826215866321]}, {'weights': [0.6953731516831658, 0.7984157678353598, 0.6222254337970721, 0.9295621474569257, 0.585316264004065, 0.558849250602328]}, {'weights': [0.3514399262838862, 0.32521320392004793, 0.36164817541322536, 0.9109892325602444, 0.9830976313748846, 0.2822166507790024]}, {'weights': [0.6562126007758324, 0.22987158411711028, 0.7560179190017289, 0.5392768129965558, 0.014816217544778909, 0.1000413808973204]}, {'weights': [0.03043504022903

In [30]:
print('Predict:', network.forward_propagate( X[7], ['relu', 'relu', 'identity']))
print('True:', y[7])

Predict: [0.9905789985877751]
True: [0.99503707155059]
